In [ ]:
!wget -O /content/drive/MyDrive/GoogleNews-vectors-negative300.bin "https://figshare.com/ndownloader/files/10798046"


In [3]:
import pandas as pd
import nltk
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer
from sklearn.metrics.pairwise import cosine_similarity

model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)

# Загрузка предобученной модели Word2Vec
# Загрузка списка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Создание токенизатора для многословных выражений
mwe_tokenizer = MWETokenizer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
ph_c = pd.read_parquet("/content/test_df_predicted_categories.parquet")

In [6]:
# Функция для получения среднего эмбеддинга для категорий
def get_average_embedding(categories):
    embeddings = []

    for category in categories:
        # Используем MWE Tokenizer для обработки многословных выражений
        tokens = mwe_tokenizer.tokenize(category)

        # Удаление стоп-слов
        filtered_words = [word for word in tokens if word.lower() not in stop_words]

        for word in filtered_words:
            try:
                embedding = model[word]
                embeddings.append(embedding)
            except KeyError:
                print(f"'{word}' not found in the vocabulary.")

    if embeddings:
        # Возвращаем средний эмбеддинг
        return np.mean(embeddings, axis=0)
    return None

ph_c['average_embedding'] = ph_c['predicted_top_categories'].apply(get_average_embedding)


' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not found in the vocabulary.
' ' not fo

In [7]:
def compute_average_cosine_distance(user_group):
    embeddings = np.vstack(user_group['average_embedding'].dropna().values)  # Получаем эмбеддинги
    if embeddings.shape[0] < 2:
      return np.nan  # Не хватает данных для вычисления расстояния

    # Вычисляем косинусные расстояния
    distances = abs(cosine_similarity(embeddings))
    # Берем только верхний треугольник матрицы расстояний, исключая диагональ
    upper_triangle = distances[np.triu_indices(len(distances), k=1)]

    return upper_triangle.mean()

# Вычисление среднего косинусного расстояния для каждого user_id
average_distances = ph_c.groupby('user_id', as_index=False, group_keys=False).apply(compute_average_cosine_distance).reset_index()

average_distances.drop(columns=['index'], inplace=True)
# Переименуем столбцы
average_distances.columns = ['user_id', 'average_distance']

# Объединяем результаты с исходным DataFrame
ph_c = ph_c.merge(average_distances, on='user_id', how='left')


<ipython-input-7-f1d5626bbeab>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  average_distances = ph_c.groupby('user_id', as_index=False, group_keys=False).apply(compute_average_cosine_distance).reset_index()


In [10]:
ph_c

,id,user_id,igm2txt,url,like_count,predicted_top_categories,average_embedding,average_distance
0,1,357,female character in a red dress surrounded by...,https://sun9-65.userapi.com/s/v1/if1/wwgdoaSH3...,104,"[anime, beauty of appearance, sculpture]","[-0.13862304, 0.1046875, 0.008447265, 0.158923...",0.945545
1,2,357,vampire holding a knife,https://sun9-8.userapi.com/s/v1/if1/gssFXziHQw...,104,"[sewing, sculpture, mysticism]","[-0.17363994, 0.08621725, -0.0063069663, 0.139...",0.945545
2,3,357,wo girls in a red chair with a butterfly on th...,https://sun9-24.userapi.com/s/v1/if1/Od_UayMZi...,107,"[teen problems, parties, beauty of appearance]","[-0.12528853, 0.12002841, -0.0119018555, 0.186...",0.945545
3,4,357,girl with long hair and a black dress,https://sun9-68.userapi.com/s/v1/if1/0x65XShbd...,116,"[beauty of appearance, sewing, anime]","[-0.1357386, 0.08487118, -0.011409984, 0.18564...",0.945545
4,5,357,girl in a black dress with a red caper,https://sun9-48.userapi.com/s/v1/if1/0ft6LUT4a...,133,"[beauty of appearance, anonymous, sewing]","[-0.13922797, 0.07271661, -0.003329807, 0.1744...",0.945545
...,...,...,...,...,...,...,...,...
2140,2141,505,man sitting on a bench in the woods,https://sun9-11.userapi.com/s/v1/ig2/J-1J5-Qdy...,5,"[outdoor activities, gardening, meditation]","[-0.14820744, 0.0640118, 0.03768217, 0.1399489...",0.945434
2141,2142,506,woman taking a self self self self self self ...,https://sun9-64.userapi.com/s/v1/ig2/k9S5yp0P0...,25,"[self-development, diy (do it yourself), beaut...","[-0.10932805, 0.124046914, 0.01763916, 0.14127...",0.986888
2142,2143,506,woman with long blonde hair,https://sun9-27.userapi.com/s/v1/ig2/JojIvhUM8...,32,"[beauty of appearance, health, sewing]","[-0.15236945, 0.12033884, -0.0152716385, 0.157...",0.986888
2143,2144,506,woman in a black jacket standing on a hill in...,https://sun9-20.userapi.com/s/v1/ig2/N99znrBju...,61,"[beauty of appearance, nature photo, fitness]","[-0.14006042, 0.109198, 0.0063903807, 0.170544...",0.986888


In [9]:
ph_c.to_parquet('/content/drive/MyDrive/ph_c_dataset.parquet', index=False)  # index=False, чтобы не сохранять индексы
